In [207]:
def verify_board_state(board_state):
    # Check the overall format
    if not isinstance(board_state, str) or not '/' in board_state:
        return False
    
    try:
        # Splitting the string to extract pawn positions for each player
        players = board_state.split('/')
        if len(players) != 2:
            return False
        
        # Extracting positions
        p1_pos = players[0].split(':')[1].split(',')
        p2_pos = players[1].split(':')[1].split(',')
        
        # Combining all positions for overlap checking, except starting point 0
        all_positions = set(p1_pos + p2_pos) - {'0'}
        
        # Verifying each player has exactly 2 pawns
        if len(p1_pos) != 2 or len(p2_pos) != 2:
            return False
        
        # Checking positions are integers, within range, and no illegal overlap
        for pos in p1_pos + p2_pos:
            pos_int = int(pos)  # This will raise ValueError if not an integer
            if pos_int < 0 or pos_int > 101 or (p1_pos.count(pos) + p2_pos.count(pos) > 1 and pos_int != 0):
                return False
        
    except ValueError:
        # Catching exceptions for invalid integer conversion and index errors
        return False
    
    return True

# Example usage
print(verify_board_state("p1:6,6/p2:0,0"))  # Expected: True, valid input
print(verify_board_state("p1:0,0/p2:0,0"))  # Expected: True, valid input
print(verify_board_state("p1:0,102/p2:4,16"))  # Expected: False, one pawn out of range
print(verify_board_state("p1:0,3/p2:3,16"))  # Expected: False, overlapping positions
print(verify_board_state("p1:-1,3/p2:4,16"))  # Expected: False, negative position


False
True
False
False
False


In [208]:
def generate_valid_moves_with_turn(board_state, roll1, roll2, player_turn=1):
    def parse_positions(state):
        positions = state.split('/')
        p1_positions = list(map(int, positions[0].split(':')[1].split(',')))
        p2_positions = list(map(int, positions[1].split(':')[1].split(',')))
        return p1_positions, p2_positions

    def calculate_moves(pos, roll, current_positions, other_positions, pos_index):
        moves = []
        operations = [('+', lambda x, y: x + y), ('-', lambda x, y: x - y),
                      ('*', lambda x, y: x * y), ('/', lambda x, y: x // y if y != 0 and x % y == 0 else None)]
    
        for symbol, operation in operations:
            result = operation(pos, roll)
            if result is not None and 0 <= result <= 101:
                new_positions = current_positions[:]
                new_positions[pos_index] = result
                
                # Check if the new position is occupied by the opponent, except for safe zones (0 and 101)
                if result in other_positions and result not in [0, 101]:
                    # Move the opponent pawn back to 0
                    other_positions = [0 if pos == result else pos for pos in other_positions]
                
                if player_turn == 1:
                    new_state = f"p1:{','.join(map(str, new_positions))}/p2:{','.join(map(str, other_positions))}"
                else:
                    new_state = f"p1:{','.join(map(str, other_positions))}/p2:{','.join(map(str, new_positions))}"
                moves.append(f"{new_state}")
        return moves


    p1_positions, p2_positions = parse_positions(board_state)
    current_positions, other_positions = (p1_positions, p2_positions) if player_turn == 1 else (p2_positions, p1_positions)
    
    valid_moves = []
    for index, pos in enumerate(current_positions):
        # Skip generating moves for the second pawn at 0 if both pawns are at 0
        if current_positions.count(0) == 2 and index == 1:
            continue
        valid_moves.extend(calculate_moves(pos, roll1, current_positions, other_positions, index))
        if roll2 is not None:
            valid_moves.extend(calculate_moves(pos, roll2, current_positions, other_positions, index))

    return set(list(valid_moves))

# Example usage
board_state = "p1:0,0/p2:0,0"
roll1, roll2 = 6, 4 # Can just have roll1 too
player_turn = 1  # Player 1's turn

valid_moves = generate_valid_moves_with_turn(board_state, roll1, roll2, player_turn)

for move in valid_moves:
    print(move)

p1:0,0/p2:0,0
p1:4,0/p2:0,0
p1:6,0/p2:0,0


In [209]:
import random

# This assumes `generate_valid_moves_with_turn` correctly appends formatted moves to `valid_moves`.
# Make sure each move string follows the "p1:x,y/p2:a,b" format.

def h_max(valid_moves, player_positions):
    max_sum = -1
    best_move = None
    for move in valid_moves:
        # Splitting move string into components for p1 and p2
        parts = move.split('/')
        p1_pos_str, p2_pos_str = parts[0], parts[1]
        p1_positions = [int(x) for x in p1_pos_str.split(':')[1].split(',')]
        p2_positions = [int(x) for x in p2_pos_str.split(':')[1].split(',')]

        current_positions = p1_positions if player_positions == "p1" else p2_positions

        positions_sum = sum(current_positions)
        if positions_sum > max_sum:
            max_sum = positions_sum
            best_move = move
    return best_move
    
def h_max_diff(valid_moves, player_positions):
    max_difference = float('-inf')
    best_move = None
    for move in valid_moves:
        # Splitting the move string to extract positions for p1 and p2
        parts = move.split('/')
        p1_pos_str, p2_pos_str = parts[0], parts[1]
        p1_positions = [int(x) for x in p1_pos_str.split(':')[1].split(',')]
        p2_positions = [int(x) for x in p2_pos_str.split(':')[1].split(',')]

        # Calculating the sum of positions for both players
        sum_p1_positions = sum(p1_positions)
        sum_p2_positions = sum(p2_positions)

        # Determining the current and opponent positions based on the player
        if player_positions == "p1":
            current_sum = sum_p1_positions
            opponent_sum = sum_p2_positions
        else:
            current_sum = sum_p2_positions
            opponent_sum = sum_p1_positions

        # Calculating the difference between the player's and the opponent's sums
        difference = current_sum - opponent_sum

        # Maximizing the difference
        if difference > max_difference:
            max_difference = difference
            best_move = move

    return best_move

In [210]:
class PrimeClimbGame:
    def __init__(self):
        self.reset_game()

    def reset_game(self):
        self.board_state = "p1:0,0/p2:0,0"
        self.current_player_turn = 1  # 1 for Player 1, 2 for Player 2
        self.winner = None
        self.move_count = 0  # Track the number of moves made

    def get_state_vector(self):
        # Extracting pawn positions from the board state
        p1_positions_str, p2_positions_str = self.board_state.split('/')[0].split(':')[1], self.board_state.split('/')[1].split(':')[1]
        p1_positions = [int(pos) for pos in p1_positions_str.split(',')]
        p2_positions = [int(pos) for pos in p2_positions_str.split(',')]
        
        # Optionally, add the current player turn as part of the state
        player_turn_vector = [1, 0] if self.current_player_turn == 1 else [0, 1]
        
        # Concatenating all parts into a single state vector
        state_vector = p1_positions + p2_positions + player_turn_vector
        return state_vector
    
    def roll_dice(self):
        return random.randint(1, 10), random.randint(1, 10)

    def switch_player(self):
        self.current_player_turn = 2 if self.current_player_turn == 1 else 1
        self.move_count += 1  # Increment move count each time players switch

    def check_winner(self):
        positions = self.board_state.split('/')
        p1_positions = list(map(int, positions[0].split(':')[1].split(',')))
        p2_positions = list(map(int, positions[1].split(':')[1].split(',')))
        if all(pos == 101 for pos in p1_positions) or all(pos == 101 for pos in p2_positions):
            self.winner = self.current_player_turn
            # print(f"Game Over! Player {self.winner} wins the game after {self.move_count} turns.")
        elif self.move_count >= 200:  # Check for tie condition
            self.winner = "Tie"
            print(f"Game Over! The game is a tie after {self.move_count} turns.")

    def play_turn(self):
        if self.winner:  # Exit if the game has already concluded
            return

        roll1, roll2 = self.roll_dice()

        try:
            valid_moves = generate_valid_moves_with_turn(self.board_state, roll1, roll2, self.current_player_turn)
            if not valid_moves:  # If no valid moves are available, the current player loses
                self.winner = 2 if self.current_player_turn == 1 else 1
                print(f"Game Over! Player {self.winner} wins by default after {self.move_count} turns.")
                return

            # Heuristic to pick the best move
            player_positions = "p1" if self.current_player_turn == 1 else "p2"
            best_move = heuristic_max(valid_moves, player_positions)
            self.board_state = best_move

            self.check_winner()
        except Exception as e:
            self.winner = 2 if self.current_player_turn == 1 else 1
            print(f"Game Over! Player {self.winner} wins by default after {self.move_count} turns due to an error.")
            return

        if not self.winner:  # Only switch players if the game hasn't concluded
            self.switch_player()

# Example of running the game
game = PrimeClimbGame()
while not game.winner:
    game.play_turn()


In [211]:
import random

def play_tournament(heuristic1, heuristic2, num_matches=1000):
    wins_p1 = 0
    wins_p2 = 0
    total_turns = 0
    turns_per_win_p1 = []
    turns_per_win_p2 = []
    ties = 0

    for _ in range(num_matches):
        game = PrimeClimbGame()
        while not game.winner and game.move_count < 200:
            roll1, roll2 = game.roll_dice()
            valid_moves = generate_valid_moves_with_turn(game.board_state, roll1, roll2, game.current_player_turn)
            
            # Select heuristic based on current player
            current_heuristic = heuristic1 if game.current_player_turn == 1 else heuristic2
            
            # Pick best move
            best_move = current_heuristic(valid_moves, f"p{game.current_player_turn}")
            game.board_state = best_move
            game.check_winner()
            game.switch_player()

        total_turns += game.move_count
        
        if game.winner == "Tie":
            ties += 1
        elif game.winner == 1:
            wins_p1 += 1
            turns_per_win_p1.append(game.move_count)
        elif game.winner == 2:
            wins_p2 += 1
            turns_per_win_p2.append(game.move_count)

    avg_turns_per_game = total_turns / num_matches
    avg_turns_per_win_p1 = sum(turns_per_win_p1) / len(turns_per_win_p1) if turns_per_win_p1 else 0
    avg_turns_per_win_p2 = sum(turns_per_win_p2) / len(turns_per_win_p2) if turns_per_win_p2 else 0

    print(f"Matches Played: {num_matches}")
    print(f"Wins Player 1 (Heuristic 1): {wins_p1}")
    print(f"Wins Player 2 (Heuristic 2): {wins_p2}")
    print(f"Ties: {ties}")
    print(f"Average Turns per Game: {avg_turns_per_game:.2f}")
    print(f"Average Turns per Win for Player 1: {avg_turns_per_win_p1:.2f}")
    print(f"Average Turns per Win for Player 2: {avg_turns_per_win_p2:.2f}")

play_tournament(h_max_diff, h_max)


Matches Played: 1000
Wins Player 1 (Heuristic 1): 811
Wins Player 2 (Heuristic 2): 175
Ties: 0
Average Turns per Game: 67.97
Average Turns per Win for Player 1: 67.16
Average Turns per Win for Player 2: 61.14
